In [283]:
#!/usr/bin/env python3.7

# Copyright 2022, Gurobi Optimization, LLC

# This example formulates and solves the following simple SOCP model:
#  minimize    t0+t1+t2
#  subject to  x^2 + y^2 <= t0^2 (second-order cone)
#              (x-1)^2 + y^2 <= t1^2 (second-order cone)
#              (x-1/2)^2 + (y-sqrt(3)/2)^2 <= t2^2 (second-order cone)
#              t0, t1, t2 non-negative

$$
\begin{array}{rl}
\min & w_0t_0+w_1t_1+w_2t_2\\
s.t. & x^2 + y^2 \leq (t_0)^2 \\
     & \sqrt{x^2 + y^2} \leq t_0 \\
     & (x-1)^2 + y^2 \leq (t_1)^2 \\
     & \left(x-\frac{1}{2}\right)^2 + \left(y-\frac{\sqrt{3}}{2}\right)^2 \leq (t_2)^2\\
     & t_0,t_1,t_2\geq 0
\end{array}
$$
The solution should be
$$
\left(\frac{1}{2},\frac{\sqrt{3}}{6}\right)
$$

In [284]:
import math
import gurobipy as gp
from gurobipy import GRB

In [285]:
#DataSet與weight由題目提供，更動此處即可求出(a)(b)(c)的最佳值

# # (a) 
# DataSet=[[0,0],[1,0],[1/2,math.sqrt(3)/2]]
# weight=[1,1,1]
# (b) 
# DataSet=[[0,0],[1,0],[0,1],[1,1]]
# weight=[1,1,1,1]
# (c)
DataSet=[[0,0,0],[1,0,0],[0,1,0],[0,0,1],[1,1,0],[1,0,1],[0,1,1],[1,1,1]]
weight=[1,1,1,1,1,1,1,1]
I = len(DataSet)#使用len()來判斷DataSets內有幾組list

In [286]:
# Create a new model
m = gp.Model("TheWeberProblem")

In [287]:
# 建立變數，使用if,else來判斷維度DataSets的維度
if len (DataSet[0]) <= 2:
    x = m.addVar(name="x")
    y = m.addVar(name="y")
else:
    x = m.addVar(name="x")
    y = m.addVar(name="y")
    z = m.addVar(name="z")
t = {}
for i in range(0,I):
    t[i] = m.addVar(name="t%s"%i)
m.update()

In [288]:
obj = 0
for i in range(0, I):#根據DataSets和weight的資料來建立目標式
    a = obj
    obj = a + weight[i]*t[i]
m. setObjective(obj, GRB.MINIMIZE)

In [289]:
if len(DataSet[0]) <=2:#根據DataSets和weight的資料來建立限制式
    for i in range(0,I):
        m.addConstr((x-DataSet[i][0])**2 + (y-DataSet[i][1])**2 <= t[i]**2, "qc%s"%i)
else:
    for i in range(0,I):
        m.addConstr((x-DataSet[i][0])**2 + (y-DataSet[i][1])**2 + (z-DataSet[i][2])**2 <= t[i]**2, "qc%s"%i)
m.params.NonConvex=2


Set parameter NonConvex to value 2


In [290]:
m.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 0 rows, 11 columns and 0 nonzeros
Model fingerprint: 0x4a491090
Model has 8 quadratic constraints
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [2e+00, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
  QRHS range       [1e+00, 3e+00]

Continuous model is non-convex -- solving as a MIP

Presolve time: 0.00s
Presolved: 14 rows, 22 columns, 54 nonzeros
Presolved model has 4 quadratic constraint(s)
Presolved model has 7 bilinear constraint(s)
Variable types: 22 continuous, 0 integer (0 binary)

Root relaxation: objective 0.000000e+00, 3 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd

In [291]:
m.getVars()

[<gurobi.Var x (value 0.5000000005038967)>,
 <gurobi.Var y (value 0.5000000005039231)>,
 <gurobi.Var z (value 0.4999999976506756)>,
 <gurobi.Var t0 (value 0.8660254030099206)>,
 <gurobi.Var t1 (value 0.8660254024264304)>,
 <gurobi.Var t2 (value 0.8660254024279224)>,
 <gurobi.Var t3 (value 0.8660254057224088)>,
 <gurobi.Var t4 (value 0.8660254018463323)>,
 <gurobi.Var t5 (value 0.8660254051386337)>,
 <gurobi.Var t6 (value 0.8660254051407453)>,
 <gurobi.Var t7 (value 0.8660254045564338)>]

In [292]:
for v in m.getVars():
    print('%s %g' % (v.VarName, v.X))

print('Obj: %g' % obj.getValue())

x 0.5
y 0.5
z 0.5
t0 0.866025
t1 0.866025
t2 0.866025
t3 0.866025
t4 0.866025
t5 0.866025
t6 0.866025
t7 0.866025
Obj: 6.9282
